# Control System Volume using thumb and indexfinger

##  Features of this Code:
✔ Smooth volume control
✔ Works in real-time
✔ Uses hand landmarks efficiently
✔ Optimized for performance

## How It Works
1. Tracks hand landmarks using MediaPipe.
2. Detects the distance between the thumb tip and index finger tip.
3. Maps the distance to system volume range.
4. Controls volume in real-time based on finger separation.
5. Displays volume level on the screen.

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Initialize Pycaw for system volume control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Get volume range
vol_range = volume.GetVolumeRange()
min_vol, max_vol = vol_range[:2]  # Volume range in decibels

# Start webcam capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Flip image for natural view
    frame = cv2.flip(frame, 1)
    
    # Convert to RGB for MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get landmark positions
            landmarks = hand_landmarks.landmark

            # Get thumb tip and index finger tip coordinates
            thumb_tip = landmarks[4]  # Thumb tip
            index_tip = landmarks[8]  # Index finger tip

            # Convert to pixel coordinates
            h, w, _ = frame.shape
            x1, y1 = int(thumb_tip.x * w), int(thumb_tip.y * h)
            x2, y2 = int(index_tip.x * w), int(index_tip.y * h)

            # Calculate distance between thumb and index finger
            length = math.hypot(x2 - x1, y2 - y1)

            # Convert distance to volume level
            vol = np.interp(length, [20, 200], [min_vol, max_vol])
            volume.SetMasterVolumeLevel(vol, None)

            # Draw indicators
            cv2.circle(frame, (x1, y1), 10, (255, 0, 0), -1)
            cv2.circle(frame, (x2, y2), 10, (255, 0, 0), -1)
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

            # Display volume percentage
            vol_percentage = np.interp(vol, [min_vol, max_vol], [0, 100])
            cv2.putText(frame, f"Volume: {int(vol_percentage)}%", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show output frame
    cv2.imshow("Volume Control", frame)

    # Exit on 'q' key
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
